In [1]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ast
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.agent import predict_position
from src.update_files import update_state, update_portfolio
from src.position_sizing import portfolio_calculator, amount_calculator
from src.risk_management import slippage, stop_loss, take_profit
from src.backtester import data_handler, place_order, execute_order, execute_SL_TP
from src.utils import get_config, read_file

In [4]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

['ETH/USDT',
 'BTC/USDT',
 'BNB/USDT',
 'SOL/USDT',
 'ADA/USDT',
 'LINK/USDT',
 'XRP/USDT',
 'LTC/USDT',
 'XLM/USDT']

In [5]:
test_data = read_file.read_merged_test_data()
test_data

,"('open', 'ETH')","('high', 'ETH')","('low', 'ETH')","('close', 'ETH')","('volume', 'ETH')","('rsi', 'ETH')","('sma-50', 'ETH')","('sma-100', 'ETH')","('sma-200', 'ETH')","('ema-50', 'ETH')",...,"('volume', 'XLM')","('rsi', 'XLM')","('sma-50', 'XLM')","('sma-100', 'XLM')","('sma-200', 'XLM')","('ema-50', 'XLM')","('ema-100', 'XLM')","('ema-200', 'XLM')","('atr', 'XLM')","('adx', 'XLM')"
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-01 00:00:00,1822.43,1843.00,1817.88,1831.94,22628.8655,54.115416,1820.6818,1857.5496,1950.81190,1829.805041,...,1765300.0,45.743670,0.266568,0.269940,0.279694,0.266534,0.270316,0.274679,0.003104,26.188115
2025-04-01 01:00:00,1831.93,1833.33,1821.03,1826.38,11413.5415,51.610623,1820.6478,1855.6334,1949.97905,1829.670726,...,708030.0,44.743440,0.266486,0.269680,0.279606,0.266403,0.270175,0.274565,0.003040,25.656347
2025-04-01 02:00:00,1826.38,1839.39,1824.39,1839.38,10437.1863,56.661616,1820.8970,1853.9491,1949.18935,1830.051482,...,2436534.0,53.037477,0.266466,0.269452,0.279535,0.266395,0.270097,0.274482,0.003051,24.084246
2025-04-01 03:00:00,1839.38,1841.10,1833.06,1835.00,10613.2322,54.593921,1820.8968,1852.2625,1948.38285,1830.245541,...,1027161.0,50.069816,0.266394,0.269239,0.279444,0.266344,0.269998,0.274388,0.003019,22.537193
2025-04-01 04:00:00,1835.00,1840.53,1831.95,1838.13,7754.7126,55.834253,1820.8806,1850.5950,1947.61140,1830.554736,...,639133.0,52.166096,0.266302,0.269030,0.279368,0.266327,0.269916,0.274304,0.002946,20.964735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30 19:00:00,4116.53,4170.43,4108.82,4160.10,29817.4828,52.843105,4145.0592,4078.9141,4085.20350,4133.753253,...,1863177.0,54.772711,0.367300,0.363290,0.363616,0.365448,0.364677,0.366572,0.003319,21.109871
2025-09-30 20:00:00,4160.11,4197.44,4148.57,4195.36,21033.0228,60.208027,4148.1216,4080.9596,4085.21815,4136.169204,...,1498663.0,57.532170,0.367436,0.363406,0.363660,0.365591,0.364765,0.366597,0.003267,21.030137
2025-09-30 21:00:00,4195.37,4201.52,4152.48,4152.74,19841.3808,50.035730,4150.3874,4082.0399,4084.94635,4136.819039,...,1966327.0,50.266007,0.367500,0.363455,0.363675,0.365600,0.364785,0.366589,0.003320,20.503979


In [6]:
update_state.set_state(test_data, 100000)
state = read_file.read_state()
state

{'timestep': '2025-04-01 00:00:00',
 'cash': 100000,
 'returns': 0,
 'drawdown': 0,
 'sharpe ratio': 0}

In [7]:
candle = data_handler.fetch_data(test_data)
candle

{'ADA': {'adx': 23.708393836565573,
  'atr': 0.0096069381100175,
  'close': 0.6626,
  'ema-100': 0.6777990988845789,
  'ema-200': 0.6949884410082713,
  'ema-50': 0.663298635361842,
  'high': 0.668,
  'low': 0.6599,
  'open': 0.6618,
  'rsi': 55.00230368210548,
  'sma-100': 0.6799889999999997,
  'sma-200': 0.7081184999999985,
  'sma-50': 0.6630720000000011,
  'volume': 5694518.7},
 'BNB': {'adx': 15.551866838489929,
  'atr': 4.418509440163837,
  'close': 607.1,
  'ema-100': 609.3910162459829,
  'ema-200': 614.827326017486,
  'ema-50': 604.3541104598204,
  'high': 607.5,
  'low': 604.46,
  'open': 604.81,
  'rsi': 57.52082545654922,
  'sma-100': 610.8285999999983,
  'sma-200': 620.5904999999987,
  'sma-50': 603.182600000001,
  'volume': 4813.093},
 'BTC': {'adx': 21.45432072737509,
  'atr': 629.4983130379409,
  'close': 82651.1,
  'ema-100': 83463.11160549734,
  'ema-200': 84236.45957230532,
  'ema-50': 82756.73585007043,
  'high': 82837.38,
  'low': 82432.74,
  'open': 82550.0,
  'rsi':

In [8]:
execute_SL_TP.execute(candle)

In [9]:
candle = predict_position.predict(candle)
candle

{'ADA': {'adx': 23.708393836565573,
  'atr': 0.0096069381100175,
  'close': 0.6626,
  'ema-100': 0.6777990988845789,
  'ema-200': 0.6949884410082713,
  'ema-50': 0.663298635361842,
  'high': 0.668,
  'low': 0.6599,
  'open': 0.6618,
  'rsi': 55.00230368210548,
  'sma-100': 0.6799889999999997,
  'sma-200': 0.7081184999999985,
  'sma-50': 0.6630720000000011,
  'volume': 5694518.7,
  'fiducia': 0.1},
 'BNB': {'adx': 15.551866838489929,
  'atr': 4.418509440163837,
  'close': 607.1,
  'ema-100': 609.3910162459829,
  'ema-200': 614.827326017486,
  'ema-50': 604.3541104598204,
  'high': 607.5,
  'low': 604.46,
  'open': 604.81,
  'rsi': 57.52082545654922,
  'sma-100': 610.8285999999983,
  'sma-200': 620.5904999999987,
  'sma-50': 603.182600000001,
  'volume': 4813.093,
  'fiducia': 0.1},
 'BTC': {'adx': 21.45432072737509,
  'atr': 629.4983130379409,
  'close': 82651.1,
  'ema-100': 83463.11160549734,
  'ema-200': 84236.45957230532,
  'ema-50': 82756.73585007043,
  'high': 82837.38,
  'low': 8

In [10]:
update_portfolio.set_portfolio()
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,0.0,0.0,0.0,0.0,0.0,0.0
BTC,0.0,0.0,0.0,0.0,0.0,0.0
BNB,0.0,0.0,0.0,0.0,0.0,0.0
SOL,0.0,0.0,0.0,0.0,0.0,0.0
ADA,0.0,0.0,0.0,0.0,0.0,0.0
LINK,0.0,0.0,0.0,0.0,0.0,0.0
XRP,0.0,0.0,0.0,0.0,0.0,0.0
LTC,0.0,0.0,0.0,0.0,0.0,0.0
XLM,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
Pt = portfolio_calculator.calculate(candle)
Pt

np.float64(382508.47479854454)

In [12]:
candle = slippage.get_order_price(candle, Pt)
candle

{'ADA': {'adx': 23.708393836565573,
  'atr': 0.0096069381100175,
  'close': 0.6626,
  'ema-100': 0.6777990988845789,
  'ema-200': 0.6949884410082713,
  'ema-50': 0.663298635361842,
  'high': 0.668,
  'low': 0.6599,
  'open': 0.6618,
  'rsi': 55.00230368210548,
  'sma-100': 0.6799889999999997,
  'sma-200': 0.7081184999999985,
  'sma-50': 0.6630720000000011,
  'volume': 5694518.7,
  'fiducia': 0.1,
  'order_price': 0.6639252},
 'BNB': {'adx': 15.551866838489929,
  'atr': 4.418509440163837,
  'close': 607.1,
  'ema-100': 609.3910162459829,
  'ema-200': 614.827326017486,
  'ema-50': 604.3541104598204,
  'high': 607.5,
  'low': 604.46,
  'open': 604.81,
  'rsi': 57.52082545654922,
  'sma-100': 610.8285999999983,
  'sma-200': 620.5904999999987,
  'sma-50': 603.182600000001,
  'volume': 4813.093,
  'fiducia': 0.1,
  'order_price': 608.3142},
 'BTC': {'adx': 21.45432072737509,
  'atr': 629.4983130379409,
  'close': 82651.1,
  'ema-100': 83463.11160549734,
  'ema-200': 84236.45957230532,
  'ema

In [13]:
candle = amount_calculator.calculate(candle, Pt)
candle

{'ADA': {'adx': 23.708393836565573,
  'atr': 0.0096069381100175,
  'close': 0.6626,
  'ema-100': 0.6777990988845789,
  'ema-200': 0.6949884410082713,
  'ema-50': 0.663298635361842,
  'high': 0.668,
  'low': 0.6599,
  'open': 0.6618,
  'rsi': 55.00230368210548,
  'sma-100': 0.6799889999999997,
  'sma-200': 0.7081184999999985,
  'sma-50': 0.6630720000000011,
  'volume': 5694518.7,
  'fiducia': 0.1,
  'order_price': 0.6639252,
  'amt': np.float64(57555.6325396437)},
 'BNB': {'adx': 15.551866838489929,
  'atr': 4.418509440163837,
  'close': 607.1,
  'ema-100': 609.3910162459829,
  'ema-200': 614.827326017486,
  'ema-50': 604.3541104598204,
  'high': 607.5,
  'low': 604.46,
  'open': 604.81,
  'rsi': 57.52082545654922,
  'sma-100': 610.8285999999983,
  'sma-200': 620.5904999999987,
  'sma-50': 603.182600000001,
  'volume': 4813.093,
  'fiducia': 0.1,
  'order_price': 608.3142,
  'amt': np.float64(62.81726588826868)},
 'BTC': {'adx': 21.45432072737509,
  'atr': 629.4983130379409,
  'close': 

In [14]:
candle = stop_loss.get_stop_loss(candle)
candle

{'ADA': {'adx': 23.708393836565573,
  'atr': 0.0096069381100175,
  'close': 0.6626,
  'ema-100': 0.6777990988845789,
  'ema-200': 0.6949884410082713,
  'ema-50': 0.663298635361842,
  'high': 0.668,
  'low': 0.6599,
  'open': 0.6618,
  'rsi': 55.00230368210548,
  'sma-100': 0.6799889999999997,
  'sma-200': 0.7081184999999985,
  'sma-50': 0.6630720000000011,
  'volume': 5694518.7,
  'fiducia': 0.1,
  'order_price': 0.6639252,
  'amt': np.float64(57555.6325396437),
  'stop_price': 0.6495147928349737,
  'stop_portion': np.float64(46044.50603171496)},
 'BNB': {'adx': 15.551866838489929,
  'atr': 4.418509440163837,
  'close': 607.1,
  'ema-100': 609.3910162459829,
  'ema-200': 614.827326017486,
  'ema-50': 604.3541104598204,
  'high': 607.5,
  'low': 604.46,
  'open': 604.81,
  'rsi': 57.52082545654922,
  'sma-100': 610.8285999999983,
  'sma-200': 620.5904999999987,
  'sma-50': 603.182600000001,
  'volume': 4813.093,
  'fiducia': 0.1,
  'order_price': 608.3142,
  'amt': np.float64(62.8172658

In [15]:
candle = take_profit.get_take_profit(candle)
candle

{'ADA': {'adx': 23.708393836565573,
  'atr': 0.0096069381100175,
  'close': 0.6626,
  'ema-100': 0.6777990988845789,
  'ema-200': 0.6949884410082713,
  'ema-50': 0.663298635361842,
  'high': 0.668,
  'low': 0.6599,
  'open': 0.6618,
  'rsi': 55.00230368210548,
  'sma-100': 0.6799889999999997,
  'sma-200': 0.7081184999999985,
  'sma-50': 0.6630720000000011,
  'volume': 5694518.7,
  'fiducia': 0.1,
  'order_price': 0.6639252,
  'amt': np.float64(57555.6325396437),
  'stop_price': 0.6495147928349737,
  'stop_portion': np.float64(46044.50603171496),
  'take_price': 0.683139076220035,
  'take_portion': np.float64(46044.50603171496)},
 'BNB': {'adx': 15.551866838489929,
  'atr': 4.418509440163837,
  'close': 607.1,
  'ema-100': 609.3910162459829,
  'ema-200': 614.827326017486,
  'ema-50': 604.3541104598204,
  'high': 607.5,
  'low': 604.46,
  'open': 604.81,
  'rsi': 57.52082545654922,
  'sma-100': 610.8285999999983,
  'sma-200': 620.5904999999987,
  'sma-50': 603.182600000001,
  'volume': 4

In [16]:
order = place_order.place(candle)
order

{'ADA': {'order_amt': np.float64(57555.6325396437),
  'order_price': 0.6639252,
  'stop_price': 0.6495147928349737,
  'stop_portion': np.float64(46044.50603171496),
  'take_price': 0.683139076220035,
  'take_portion': np.float64(46044.50603171496)},
 'BNB': {'order_amt': np.float64(62.81726588826868),
  'order_price': 608.3142,
  'stop_price': 601.6864358397543,
  'stop_portion': np.float64(50.253812710614945),
  'take_price': 617.1512188803277,
  'take_portion': np.float64(50.253812710614945)},
 'BTC': {'order_amt': np.float64(0.461413848342828),
  'order_price': 82816.40220000001,
  'stop_price': 81872.1547304431,
  'stop_portion': np.float64(0.3691310786742624),
  'take_price': 84075.39882607589,
  'take_portion': np.float64(0.3691310786742624)},
 'ETH': {'order_amt': np.float64(20.817473345616076),
  'order_price': 1835.6038800000001,
  'stop_price': 1805.2422782551496,
  'stop_portion': np.float64(16.653978676492862),
  'take_price': 1876.0860156598008,
  'take_portion': np.float6

In [17]:
execute_order.execute(order)
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,20.817473,1835.603880,1805.242278,16.653979,1876.086016,16.653979
BTC,0.461414,82816.402200,81872.154730,0.369131,84075.398826,0.369131
BNB,62.817266,608.314200,601.686436,50.253813,617.151219,50.253813
SOL,306.291560,124.759020,122.325320,245.033248,128.003953,245.033248
ADA,57555.632540,0.663925,0.649515,46044.506032,0.683139,46044.506032
LINK,2806.207662,13.617180,13.330461,2244.966129,13.999472,2244.966129
XRP,18348.904023,2.082557,2.035382,14679.123218,2.145457,14679.123218
LTC,457.654652,83.496660,81.969518,366.123721,85.532850,366.123721
XLM,144675.121854,0.264127,0.259471,115740.097483,0.270336,115740.097483


In [18]:
state = read_file.read_state()
state

{'timestep': '2025-04-01 01:00:00',
 'cash': 38250.84747985441,
 'returns': 0,
 'drawdown': 0,
 'sharpe ratio': 0}

In [19]:
candle2 = data_handler.fetch_data(test_data)
candle2

{'ADA': {'adx': 22.235967599918524,
  'atr': 0.0093207282450162,
  'close': 0.6602,
  'ema-100': 0.6774506018769635,
  'ema-200': 0.694642287366398,
  'ema-50': 0.6631771202496128,
  'high': 0.6633,
  'low': 0.6577,
  'open': 0.6625,
  'rsi': 52.5585888252168,
  'sma-100': 0.6791689999999998,
  'sma-200': 0.7078979999999986,
  'sma-50': 0.6628120000000011,
  'volume': 3892829.2},
 'BNB': {'adx': 14.594540027165737,
  'atr': 4.265758765866418,
  'close': 605.69,
  'ema-100': 609.3177287955674,
  'ema-200': 614.7364073506453,
  'ema-50': 604.4064982849254,
  'high': 607.1,
  'low': 604.82,
  'open': 607.09,
  'rsi': 54.61997096820463,
  'sma-100': 610.4913999999983,
  'sma-200': 620.5035999999986,
  'sma-50': 603.2366000000009,
  'volume': 3784.169},
 'BTC': {'adx': 20.20176748917469,
  'atr': 604.6627192495167,
  'close': 82704.2,
  'ema-100': 83448.08365291324,
  'ema-200': 84221.21320840178,
  'ema-50': 82754.67562065591,
  'high': 82741.8,
  'low': 82460.0,
  'open': 82651.09,
  'rsi

In [20]:
# execute sltp
execute_SL_TP.execute(candle2)

In [21]:
candle2 = predict_position.predict(candle2)
candle2

{'ADA': {'adx': 22.235967599918524,
  'atr': 0.0093207282450162,
  'close': 0.6602,
  'ema-100': 0.6774506018769635,
  'ema-200': 0.694642287366398,
  'ema-50': 0.6631771202496128,
  'high': 0.6633,
  'low': 0.6577,
  'open': 0.6625,
  'rsi': 52.5585888252168,
  'sma-100': 0.6791689999999998,
  'sma-200': 0.7078979999999986,
  'sma-50': 0.6628120000000011,
  'volume': 3892829.2,
  'fiducia': 0.1},
 'BNB': {'adx': 14.594540027165737,
  'atr': 4.265758765866418,
  'close': 605.69,
  'ema-100': 609.3177287955674,
  'ema-200': 614.7364073506453,
  'ema-50': 604.4064982849254,
  'high': 607.1,
  'low': 604.82,
  'open': 607.09,
  'rsi': 54.61997096820463,
  'sma-100': 610.4913999999983,
  'sma-200': 620.5035999999986,
  'sma-50': 603.2366000000009,
  'volume': 3784.169,
  'fiducia': 0.1},
 'BTC': {'adx': 20.20176748917469,
  'atr': 604.6627192495167,
  'close': 82704.2,
  'ema-100': 83448.08365291324,
  'ema-200': 84221.21320840178,
  'ema-50': 82754.67562065591,
  'high': 82741.8,
  'low':

In [22]:
Pt = portfolio_calculator.calculate(candle)
Pt

np.float64(381478.10656676575)

In [23]:
candle2 = slippage.get_order_price(candle2, Pt)
candle2

{'ADA': {'adx': 22.235967599918524,
  'atr': 0.0093207282450162,
  'close': 0.6602,
  'ema-100': 0.6774506018769635,
  'ema-200': 0.694642287366398,
  'ema-50': 0.6631771202496128,
  'high': 0.6633,
  'low': 0.6577,
  'open': 0.6625,
  'rsi': 52.5585888252168,
  'sma-100': 0.6791689999999998,
  'sma-200': 0.7078979999999986,
  'sma-50': 0.6628120000000011,
  'volume': 3892829.2,
  'fiducia': 0.1,
  'order_price': 0.6615204},
 'BNB': {'adx': 14.594540027165737,
  'atr': 4.265758765866418,
  'close': 605.69,
  'ema-100': 609.3177287955674,
  'ema-200': 614.7364073506453,
  'ema-50': 604.4064982849254,
  'high': 607.1,
  'low': 604.82,
  'open': 607.09,
  'rsi': 54.61997096820463,
  'sma-100': 610.4913999999983,
  'sma-200': 620.5035999999986,
  'sma-50': 603.2366000000009,
  'volume': 3784.169,
  'fiducia': 0.1,
  'order_price': np.float64(608.3142)},
 'BTC': {'adx': 20.20176748917469,
  'atr': 604.6627192495167,
  'close': 82704.2,
  'ema-100': 83448.08365291324,
  'ema-200': 84221.2132

In [24]:
candle2 = amount_calculator.calculate(candle2, Pt)
candle2

{'ADA': {'adx': 22.235967599918524,
  'atr': 0.0093207282450162,
  'close': 0.6602,
  'ema-100': 0.6774506018769635,
  'ema-200': 0.694642287366398,
  'ema-50': 0.6631771202496128,
  'high': 0.6633,
  'low': 0.6577,
  'open': 0.6625,
  'rsi': 52.5585888252168,
  'sma-100': 0.6791689999999998,
  'sma-200': 0.7078979999999986,
  'sma-50': 0.6628120000000011,
  'volume': 3892829.2,
  'fiducia': 0.1,
  'order_price': 0.6615204,
  'amt': np.float64(57609.26035798874)},
 'BNB': {'adx': 14.594540027165737,
  'atr': 4.265758765866418,
  'close': 605.69,
  'ema-100': 609.3177287955674,
  'ema-200': 614.7364073506453,
  'ema-50': 604.4064982849254,
  'high': 607.1,
  'low': 604.82,
  'open': 607.09,
  'rsi': 54.61997096820463,
  'sma-100': 610.4913999999983,
  'sma-200': 620.5035999999986,
  'sma-50': 603.2366000000009,
  'volume': 3784.169,
  'fiducia': 0.1,
  'order_price': np.float64(608.3142),
  'amt': np.float64(62.81726588826868)},
 'BTC': {'adx': 20.20176748917469,
  'atr': 604.6627192495

In [25]:
candle2 = stop_loss.get_stop_loss(candle2)
candle2 = take_profit.get_take_profit(candle2)
candle2

{'ADA': {'adx': 22.235967599918524,
  'atr': 0.0093207282450162,
  'close': 0.6602,
  'ema-100': 0.6774506018769635,
  'ema-200': 0.694642287366398,
  'ema-50': 0.6631771202496128,
  'high': 0.6633,
  'low': 0.6577,
  'open': 0.6625,
  'rsi': 52.5585888252168,
  'sma-100': 0.6791689999999998,
  'sma-200': 0.7078979999999986,
  'sma-50': 0.6628120000000011,
  'volume': 3892829.2,
  'fiducia': 0.1,
  'order_price': 0.6615204,
  'amt': np.float64(57609.26035798874),
  'stop_price': 0.6475393076324757,
  'stop_portion': np.float64(46087.408286391),
  'take_price': 0.6801618564900324,
  'take_portion': np.float64(46087.408286391)},
 'BNB': {'adx': 14.594540027165737,
  'atr': 4.265758765866418,
  'close': 605.69,
  'ema-100': 609.3177287955674,
  'ema-200': 614.7364073506453,
  'ema-50': 604.4064982849254,
  'high': 607.1,
  'low': 604.82,
  'open': 607.09,
  'rsi': 54.61997096820463,
  'sma-100': 610.4913999999983,
  'sma-200': 620.5035999999986,
  'sma-50': 603.2366000000009,
  'volume': 

In [26]:
order2 = place_order.place(candle2)
order2

{'ADA': {'order_amt': np.float64(53.627818345041305),
  'order_price': 0.6615204,
  'stop_price': 0.6475393076324757,
  'stop_portion': np.float64(46087.408286391),
  'take_price': 0.6801618564900324,
  'take_portion': np.float64(46087.408286391)},
 'BNB': {'order_amt': np.float64(0.0),
  'order_price': np.float64(608.3142),
  'stop_price': np.float64(601.9155618512004),
  'stop_portion': np.float64(50.253812710614945),
  'take_price': np.float64(616.8457175317328),
  'take_portion': np.float64(50.253812710614945)},
 'BTC': {'order_amt': np.float64(0.0),
  'order_price': np.float64(82816.40220000001),
  'stop_price': np.float64(81909.40812112573),
  'stop_portion': np.float64(0.3691310786742624),
  'take_price': np.float64(84025.72763849904),
  'take_portion': np.float64(0.3691310786742624)},
 'ETH': {'order_amt': np.float64(0.007127059745439368),
  'order_price': 1830.03276,
  'stop_price': 1800.5219869512102,
  'stop_portion': np.float64(16.659680324289216),
  'take_price': 1869.3804

In [27]:
execute_order.execute(order2)
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,20.824600,3665.636640,3605.764265,33.313659,3745.466473,33.313659
BTC,0.461414,165632.804400,163781.562852,0.738262,168101.126465,0.738262
BNB,62.817266,1216.628400,1203.601998,100.507625,1233.996936,100.507625
SOL,306.291560,249.518040,244.717333,490.066496,255.918983,490.066496
ADA,57609.260358,1.325446,1.297054,92131.914318,1.363301,92131.914318
LINK,2806.207662,27.234360,26.668544,4489.932259,27.988781,4489.932259
XRP,18329.608363,4.161690,4.068557,29342.809908,4.285868,29342.809908
LTC,458.126159,166.682700,163.678569,732.624648,170.688209,732.624648
XLM,144675.121854,0.528254,0.519038,231480.194967,0.540542,231480.194967


In [28]:
state = read_file.read_state()
state

{'timestep': '2025-04-01 02:00:00',
 'cash': 38203.17666065398,
 'returns': 0,
 'drawdown': 0,
 'sharpe ratio': 0}